In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [3]:
dataset = load_dataset("cnn_dailymail","3.0.0", split = 'train[:1%]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
print(dataset[0].keys())

dict_keys(['article', 'highlights', 'id'])


In [5]:
dataset[0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [6]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
def preprocess_function(examples):
  inputs = ["Summarize: " + doc for doc in examples['article']]
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding = 'max_length')
  labels = tokenizer(examples['highlights'], max_length = 200, truncation = True, padding='max_length')
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [15]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

In [17]:
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 3,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    eval_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.571800,0.542388
2,0.648000,0.523090
3,0.627900,0.518105


TrainOutput(global_step=2154, training_loss=0.611484787479638, metrics={'train_runtime': 1234.0746, 'train_samples_per_second': 6.979, 'train_steps_per_second': 1.745, 'total_flos': 2331397872156672.0, 'train_loss': 0.611484787479638, 'epoch': 3.0})

In [22]:
trainer.save_model("./fine_tuned_t5_summarizer")

In [34]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "/content/fine_tuned_t5_summarizer"

try:
    fine_tuned_tokenizer = T5Tokenizer.from_pretrained(model_path)
    fine_tuned_model = T5ForConditionalGeneration.from_pretrained(model_path)
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")


Model and tokenizer loaded successfully.


In [35]:
import os

model_path = "/content/fine_tuned_t5_summarizer"

if os.path.isdir(model_path):
    print("Directory exists.")
    files = os.listdir(model_path)
    print("Files in directory:", files)
else:
    print("Directory does not exist.")


Directory exists.
Files in directory: ['training_args.bin', 'added_tokens.json', 'tokenizer_config.json', 'special_tokens_map.json', 'generation_config.json', 'model.safetensors', 'config.json', 'spiece.model']


In [36]:
# Function to generate a summary for new text
def generate_summary(text):
    input_text = "summarize: " + text
    input_ids = fine_tuned_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = fine_tuned_model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = fine_tuned_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [38]:
text = """Pichai worked in engineering and product management at Applied Materials and in management consulting at McKinsey & Company.[27] Pichai joined Google in 2004, where he led the product management and innovation efforts for a suite of Google's client software products, including Google Chrome[28] and ChromeOS, as well as being largely responsible for Google Drive. He went on to oversee the development of other applications such as Gmail and Google Maps.[29][30] On November 19, 2009, Pichai gave a demonstration of ChromeOS; the Chromebook was released for trial and testing in 2011, and released to the public in 2012.[31] On May 20, 2010, he announced the open-sourcing of the new video codec VP8 by Google and introduced the new video format, WebM.[32]

On March 13, 2013, Pichai added Android to the list of Google products that he oversaw. Android was formerly managed by Andy Rubin,[33] who was a director of Jive Software from April 2011 to July 30, 2013.[34][35][36] Pichai was selected to become the next CEO of Google on August 10, 2015,[37] after previously being appointed Product Chief by CEO, Larry Page. On October 24, 2015, he stepped into the new position at the completion of the formation of Alphabet Inc., the new holding company for the Google company family.[38][36][37]

Pichai had been suggested as a contender for Microsoft's CEO in 2014, a position that was eventually given to Satya Nadella.[39][40] In August 2017, Pichai drew publicity for firing a Google employee who wrote a ten-page manifesto criticizing the company's pro-diversity policies.[41][42][43][44][45]"""

In [39]:
# Example usage
#test_article = dataset[0]['article']

test_article = text
summary = generate_summary(test_article)
print("Original Article:", test_article)
print("Generated Summary:", summary)

Original Article: Pichai worked in engineering and product management at Applied Materials and in management consulting at McKinsey & Company.[27] Pichai joined Google in 2004, where he led the product management and innovation efforts for a suite of Google's client software products, including Google Chrome[28] and ChromeOS, as well as being largely responsible for Google Drive. He went on to oversee the development of other applications such as Gmail and Google Maps.[29][30] On November 19, 2009, Pichai gave a demonstration of ChromeOS; the Chromebook was released for trial and testing in 2011, and released to the public in 2012.[31] On May 20, 2010, he announced the open-sourcing of the new video codec VP8 by Google and introduced the new video format, WebM.[32]

On March 13, 2013, Pichai added Android to the list of Google products that he oversaw. Android was formerly managed by Andy Rubin,[33] who was a director of Jive Software from April 2011 to July 30, 2013.[34][35][36] Picha